<a href="https://colab.research.google.com/github/ms20237/3D-processing-of-2D-model/blob/main/examples/COLAB/COLAB_DEMO_mouse_openfield.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut Toolbox - Colab Demo on Topview Mouse Data
https://github.com/DeepLabCut/DeepLabCut

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1559935526258-KFYZC8BDHK01ZIDPNVIX/mouse_skel_trail.gif?format=450w)

Demo supporting: Nath\*, Mathis\* et al. *Using DeepLabCut for markerless3D  pose estimation during behavior across species. Nature Protocols, 2019

This notebook demonstrates the necessary steps to use DeepLabCut on our demo data. We provide a sub-set of the mouse data from Mathis et al, 2018 Nature Neuroscience.

This demo notebook mostly shows the most simple code to train and evaluate your model, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to use the cloud to:

- load demo data
- create a training set
- train a network
- evaluate a network
- analyze a novel video

## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"

As the COLAB environments were updated to CUDA 12.X and Python 3.11, we need to install DeepLabCut and TensorFlow in a distinct way to get TensorFlow to connect to the GPU.

In [1]:
# Install TensorFlow, tensorpack and tf_slim versions compatible with DeepLabCut
!pip install "tensorflow==2.12.1" "tensorpack>=0.11" "tf_slim>=1.1.0"

In [2]:
# Downgrade PyTorch to a version using CUDA 11.8 and cudnn 8
# This will also install the required CUDA libraries, for both PyTorch and TensorFlow
!pip install torch==2.3.1 torchvision --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 839.7/839.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.9/142.9 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# As described in https://www.tensorflow.org/install/pip#step-by-step_instructions,
# create symbolic links to NVIDIA shared libraries:
!ln -svf /usr/local/lib/python3.11/dist-packages/nvidia/*/lib/*.so* /usr/local/lib/python3.11/dist-packages/tensorflow

'/usr/local/lib/python3.11/dist-packages/tensorflow/libcublasLt.so.11' -> '/usr/local/lib/python3.11/dist-packages/nvidia/cublas/lib/libcublasLt.so.11'
'/usr/local/lib/python3.11/dist-packages/tensorflow/libcublasLt.so.12' -> '/usr/local/lib/python3.11/dist-packages/nvidia/cublas/lib/libcublasLt.so.12'
'/usr/local/lib/python3.11/dist-packages/tensorflow/libcublas.so.11' -> '/usr/local/lib/python3.11/dist-packages/nvidia/cublas/lib/libcublas.so.11'
'/usr/local/lib/python3.11/dist-packages/tensorflow/libcublas.so.12' -> '/usr/local/lib/python3.11/dist-packages/nvidia/cublas/lib/libcublas.so.12'
'/usr/local/lib/python3.11/dist-packages/tensorflow/libnvblas.so.11' -> '/usr/local/lib/python3.11/dist-packages/nvidia/cublas/lib/libnvblas.so.11'
'/usr/local/lib/python3.11/dist-packages/tensorflow/libnvblas.so.12' -> '/usr/local/lib/python3.11/dist-packages/nvidia/cublas/lib/libnvblas.so.12'
'/usr/local/lib/python3.11/dist-packages/tensorflow/libcheckpoint.so' -> '/usr/local/lib/python3.11/dist

In [4]:
# Clone the entire deeplabcut repo so we can use the demo data:
!git clone -l -s https://github.com/DeepLabCut/DeepLabCut.git cloned-DLC-repo
%cd cloned-DLC-repo
!ls

Cloning into 'cloned-DLC-repo'...
remote: Enumerating objects: 24170, done.
remote: Counting objects: 100% (2600/2600), done.
remote: Compressing objects: 100% (868/868), done.
remote: Total 24170 (delta 2187), reused 1773 (delta 1732), pack-reused 21570 (from 4)
Receiving objects: 100% (24170/24170), 189.99 MiB | 17.13 MiB/s, done.
Resolving deltas: 100% (16660/16660), done.
/content/cloned-DLC-repo
AUTHORS		    CONTRIBUTING.md  docs	 README.md	   tests
CODE_OF_CONDUCT.md  deeplabcut	     examples	 reinstall.sh	   testscript_cli.py
conda-environments  dlc.py	     LICENSE	 requirements.txt  _toc.yml
_config.yml	    docker	     NOTICE.yml  setup.py	   tools


In [5]:
%cd /content/cloned-DLC-repo/examples/openfield-Pranav-2018-10-30
!ls

/content/cloned-DLC-repo/examples/openfield-Pranav-2018-10-30
config.yaml  labeled-data  videos


In [7]:
# Install the latest DeepLabCut version (this will take a few minutes to install all the dependencies!)
%cd /content/cloned-DLC-repo/
!pip install "."

/content/cloned-DLC-repo
Processing /content/cloned-DLC-repo
  Preparing metadata (setup.py) ... done
  Created wheel for deeplabcut: filename=deeplabcut-2.3.11-py3-none-any.whl size=1547929 sha256=4fae258b0375d5d7b1be5cb742f5d87fb6ef436d553daf6e07d771acdfe32217
  Stored in directory: /root/.cache/pip/wheels/c2/3a/f9/a9c6d9217bb3a8d45f28bb0f8398ccf6f9b2908a31c788f160
Successfully built deeplabcut
  Attempting uninstall: deeplabcut
    Found existing installation: deeplabcut 2.3.11
    Uninstalling deeplabcut-2.3.11:
      Successfully uninstalled deeplabcut-2.3.11


## PLEASE, click "restart runtime" from the output above before proceeding!

In [8]:
import deeplabcut

Loading DLC 2.3.11...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [9]:
#create a path variable that links to the config file:
path_config_file = '/content/cloned-DLC-repo/examples/openfield-Pranav-2018-10-30/config.yaml'

# Loading example data set:
deeplabcut.load_demo_data(path_config_file)

Loaded, now creating training data...
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


## Start training:
This function trains the network for a specific shuffle of the training dataset.

In [10]:
#let's also change the display and save_iters just in case Colab takes away the GPU...
#if that happens, you can reload from a saved point. Typically, you want to train to 200,000 + iterations.
#more info and there are more things you can set: https://github.com/DeepLabCut/DeepLabCut/wiki/DOCSTRINGS#train_network

deeplabcut.train_network(path_config_file, shuffle=1, displayiters=100,saveiters=500, maxiters=10000)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.03M iterations).
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['snout', 'leftear', 'rightear', 'tailbase'],
 'alpha_r': 0.02,
 'apply_prob': 0.5,
 'batch_size': 1,
 'contrast': {'clahe': True,
              'claheratio': 0.1,
              'histeq': True,
              'histeqratio': 0.1},
 'convolution': {'edge': False,
                 'emboss': {'alpha': [0.0, 1.0], 'strength': [0.5, 1.5]},
                 'embossratio': 0.1,
                 'sharpen': False,
                 'sharpenratio': 0.3},
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_openfieldOct30/openfield_Pranav95shuffle1.mat',
 'dataset_type': 'imgaug',
 'decay_steps': 30000,
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/content/cloned-DLC-repo/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'loca

Selecting single-animal trainer
Batch Size is 1


/usr/local/lib/python3.11/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Loading ImageNet-pretrained resnet_50
Max_iters overwritten as 10000
Display_iters overwritten as 100
Save_iters overwritten as 500
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/content/cloned-DLC-repo/examples/openfield-Pranav-2018-10-30/dlc-models/iteration-0/openfieldOct30-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 1, 'dataset_type': 'imgaug', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfield_predict': False, 'pairwise_predict': False, 

iteration: 100 loss: 0.0601 lr: 0.005
iteration: 200 loss: 0.0236 lr: 0.005
iteration: 300 loss: 0.0207 lr: 0.005
iteration: 400 loss: 0.0201 lr: 0.005
iteration: 500 loss: 0.0174 lr: 0.005
iteration: 600 loss: 0.0160 lr: 0.005
iteration: 700 loss: 0.0149 lr: 0.005
iteration: 800 loss: 0.0142 lr: 0.005
iteration: 900 loss: 0.0131 lr: 0.005
iteration: 1000 loss: 0.0115 lr: 0.005
iteration: 1100 loss: 0.0116 lr: 0.005
iteration: 1200 loss: 0.0105 lr: 0.005
iteration: 1300 loss: 0.0093 lr: 0.005
iteration: 1400 loss: 0.0102 lr: 0.005
iteration: 1500 loss: 0.0091 lr: 0.005
iteration: 1600 loss: 0.0093 lr: 0.005
iteration: 1700 loss: 0.0090 lr: 0.005
iteration: 1800 loss: 0.0091 lr: 0.005
iteration: 1900 loss: 0.0095 lr: 0.005
iteration: 2000 loss: 0.0084 lr: 0.005
iteration: 2100 loss: 0.0089 lr: 0.005
iteration: 2200 loss: 0.0079 lr: 0.005
iteration: 2300 loss: 0.0079 lr: 0.005
iteration: 2400 loss: 0.0076 lr: 0.005
iteration: 2500 loss: 0.0078 lr: 0.005
iteration: 2600 loss: 0.0065 lr: 0

The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.


We recommend you run this for ~1,000 iterations, just as a demo. This should take around 20 min. Note, that **when you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating:
This function evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [11]:
%matplotlib notebook
deeplabcut.evaluate_network(path_config_file,plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, so be sure your labels are good!

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['snout', 'leftear', 'rightear', 'tailbase'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_openfieldOct30/openfield_Pranav95shuffle1.mat',
 'dataset_type': 'imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/content/cloned-DLC-repo/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'mirror': False,
 'net_type': 'resnet_50',
 'num_joints': 4,
 'optimizer': 'sgd',
 'pairwise_huber_loss': True,
 'pairwise_predict': False,
 'partaffinityfield_predict': False,
 'regularize': False,
 'scoremap_dir': 'test',
 'shuffle': True,
 'snapshot_prefix': '/content/cloned-D

Running  DLC_resnet50_openfieldOct30shuffle1_10000  with # of training iterations: 10000
Running evaluation ...


116it [00:07, 15.66it/s]
/content/cloned-DLC-repo/deeplabcut/pose_estimation_tensorflow/core/evaluate.py:889: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  DataMachine.to_hdf(resultsfilename, "df_with_missing")


Analysis is done and the results are stored (see evaluation-results) for snapshot:  snapshot-10000
Results for 10000  training iterations: 95 1 train error: 2.78 pixels. Test error: 4.15  pixels.
With pcutoff of 0.4  train error: 2.78 pixels. Test error: 3.14 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
Plotting...


<IPython.core.display.Javascript object>

  0%|          | 0/116 [00:00<?, ?it/s]/content/cloned-DLC-repo/deeplabcut/utils/visualization.py:67: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  DataCombined[loopscorer][bp]["y"][imagenr]
/content/cloned-DLC-repo/deeplabcut/utils/visualization.py:68: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  + DataCombined[loopscorer][bp]["x"][imagenr]
/content/cloned-DLC-repo/deeplabcut/utils/visualization.py:71: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use 

The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
Please check the results, then choose the best model (snapshot) for prediction. You can update the config.yaml file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise, consider adding more labeled-data and retraining the network (see DeepLabCut workflow Fig 2, Nath 2019)


**Check the images**:
You can go look in the newly created "evalutaion-results" folder at the images. At around 3500 iterations, the error is ~3 pixels (but this can vary on how your demo data was split for training)

## Start Analyzing videos:
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides.

**On the demo data, this should take around ~ 3 min! (The demo frames are 640x480, which should run around 35 FPS on the google-provided GPU)**

In [12]:
videofile_path = ['/content/cloned-DLC-repo/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4'] #Enter the list of videos to analyze.
deeplabcut.analyze_videos(path_config_file,videofile_path, videotype='.mp4')

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['snout', 'leftear', 'rightear', 'tailbase'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_openfieldOct30/openfield_Pranav95shuffle1.mat',
 'dataset_type': 'imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/content/cloned-DLC-repo/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'mirror': False,
 'net_type': 'resnet_50',
 'num_joints': 4,
 'optimizer': 'sgd',
 'pairwise_huber_loss': True,
 'pairwise_predict': False,
 'partaffinityfield_predict': False,
 'regularize': False,
 'scoremap_dir': 'test',
 'shuffle': True,
 'snapshot_prefix': '/content/cloned-D

Using snapshot-10000 for model /content/cloned-DLC-repo/examples/openfield-Pranav-2018-10-30/dlc-models/iteration-0/openfieldOct30-trainset95shuffle1
Starting to analyze %  /content/cloned-DLC-repo/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4
Loading  /content/cloned-DLC-repo/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4
Duration of video [s]:  77.67 , recorded with  30.0 fps!
Overall # of frames:  2330  found with (before cropping) frame dimensions:  640 480
Starting to extract posture


100%|██████████| 2330/2330 [00:56<00:00, 40.88it/s]
/content/cloned-DLC-repo/deeplabcut/utils/auxiliaryfunctions.py:403: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  DataMachine.to_hdf(dataname, "df_with_missing", format="table", mode="w")


Saving results in /content/cloned-DLC-repo/examples/openfield-Pranav-2018-10-30/videos...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


'DLC_resnet50_openfieldOct30shuffle1_10000'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Create labeled video:
This function is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. This should run around 215 FPS on the demo video!

In [13]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

Starting to process video: /content/cloned-DLC-repo/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4
Loading /content/cloned-DLC-repo/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4 and data.
Duration of video [s]: 77.67, recorded with 30.0 fps!
Overall # of frames: 2330 with cropped frame dimensions: 640 480
Generating frames and creating video.


100%|██████████| 2330/2330 [00:15<00:00, 149.51it/s]


[True]

## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [14]:
deeplabcut.plot_trajectories(path_config_file,videofile_path)

Loading  /content/cloned-DLC-repo/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4 and data.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Plots created! Please check the directory "plot-poses" within the video directory
